<a href="https://colab.research.google.com/github/nameanonymous/GoogleColabwithDB/blob/main/Dreambooth_in_Colab_AiO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DreamboothをColabで全て完結させるためのノートブックです。
モデルの入出力を全てGoogleDriveに委ねているので、現在持っているモデルを先にDriveに入れておくといいです。

まず今の状態でちゃんとGPUらが動くかどうかを確認します。

In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15109 MiB, 15109 MiB


## ckpt to Diffusers conversion


まずは現在持っているckptモデルをdiffusers形式に変換します。

In [ ]:
%pip install -q diffusers transformers omegaconf

     |████████████████████████████████| 255 kB 9.5 MB/s 
     |████████████████████████████████| 5.3 MB 31.1 MB/s 
     |████████████████████████████████| 79 kB 4.3 MB/s 
     |████████████████████████████████| 163 kB 70.9 MB/s 
     |████████████████████████████████| 7.6 MB 67.6 MB/s 
     |████████████████████████████████| 117 kB 67.1 MB/s 


Google Driveに接続します。したくない場合は共有リンクを作成しgdownする方法もありますが、正直手間です。Driveにあるフォルダはcontent/drive/MyDrive内にあります。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


自分が置いたckptファイルのパスを指定します。フォルダを先に作っておき、そこの中に入れておけば今後複数回DBをする際にファイルらがとっ散らかずに済みますのでDrive内にフォルダを作成しておくといいかもしれません。またyaml設定ファイルも忘れずにインポートしておきましょう。

In [ ]:
#@markdown ckptファイルのパス
MODEL_PATH = "/content/drive/MyDrive/***" #@param {type:"string"}

#@markdown yamlファイルのパス
YAML_PATH = "/content/drive/MyDrive/***" #@param {type:"string"}

Diffusersモデル変換に必要なコードをクローンします。

In [ ]:
!git clone https://github.com/huggingface/diffusers

In [ ]:
%cd /content/diffusers/scripts

変換します。Diffusionフォルダが作成され、そのフォルダが変換されたものとなります。このパス(/content/diffusion)は後で使いますのでよく覚えておきましょう。

In [ ]:
!python convert_original_stable_diffusion_to_diffusers.py --checkpoint_path=$MODEL_PATH --original_config_file=$YAML_PATH  --dump_path /content/diffusion

追記:
因みに出来たdiffusersモデルをzip形式で圧縮することによってダウンロードすることも可能です。

In [ ]:
#@markdown ダウンロードファイル名(zip)
DOWNLOAD_FILE_NAME = "ABCDE" #@param {type:"string"}
DOWNLOAD_PATH = "/content/" + DOWNLOAD_FILE_NAME + ".zip"

In [ ]:
!zip -r $DOWNLOAD_PATH /content/diffusion

	zip warning: name not matched: /content/diffusion

zip error: Nothing to do! (try: zip -r /content/ABCDE.zip . -i /content/diffusion)


## 学習させる

学習させるための環境創りをします。

In [ ]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate==0.12.0 transformers ftfy bitsandbytes gradio
%pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 8.3 MB/s 
     |████████████████████████████████| 18.7 MB 136 kB/s 
     |████████████████████████████████| 143 kB 6.9 MB/s 
     |████████████████████████████████| 5.3 MB 53.7 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 62.5 MB 1.2 MB/s 
     |████████████████████████████████| 5.3 MB 53.7 MB/s 
     |████████████████████████████████| 7.6 MB 41.3 MB/s 
     |████████████████████████████████| 212 kB 63.2 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 54 kB 3.6 MB/s 
     |████████████████████████████████| 112 kB 76.8 MB/s 
     |████████████████████████████████| 2.3 MB 46.2 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 270 kB 69.2 MB/s 
     |████████

設定をします。先程のdiffusionフォルダをここで指定します(デフォルトの場合MODEL NAME欄は弄らなくて大丈夫です)。また512*512にサイズ調整した画像が置いてあるフォルダもここで指定します。

 

> 注意:CLASS NAMEについて


今回DBで覚えさせる新しい概念新しい概念(キャラ、ポーズ、他)「の」プロンプトを入力してください。
以下実行する際の引数にINSTANCE PROMPT と CLASS PROMPTがあると思われますが、
これは簡単に言うのであれば学習させるものがポーズであれば「AがBする」というプロンプトと「A」と言うプロンプトの差を学習させることになるため(結果その差が「Bする」というものになって学習がされます)、もしあまりにもそれぞれのプロンプトが近すぎる（もしくは遠すぎる）場合学習が上手くされない場合があります。動かすデータセットの中身を確認しながらいじるようにして下さい。よくわからないのであればデフォルトのままでも大丈夫だとは思います。
キャラを覚えさせたい場合はCLASS PROMPTににそのキャラが女の子の場合はgirl、男の子の場合はboyを入れてみるといいかもしれません。(NAIモデルだとモデルだと1girlとかの方がいいかもとかの方がいいかも)

**ちなみにインスタンスとクラス双方全く同じプロンプトにしてしまうと「そもそものデータセットが壊れます」！**

In [ ]:
#@markdown　もしモデルを直接GoogleDriveに保存したいのであればチェックを入れてください。
save_to_gdrive = True #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

#@markdown diffusersモデルがある指定します
MODEL_NAME = "/content/drive/MyDrive/diffusion" #@param {type:"string"}

#@markdown 学習させたいイメージフォルダのパス
INSTANCE_DIR = "/content/drive/MyDrive/iloveimg-resized" #@param {type:"string"}
!mkdir -p $INSTANCE_DIR

#@markdown ↑のイメージで学習させたいプロンプト
CLASS_NAME = "yui" #@param {type:"string"}
CLASS_DIR = f"/content/data/{CLASS_NAME}"

#@markdown 学習後のモデルの保存先.

OUTPUT_DIR = "stable_diffusion_weights/sks" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*]  {OUTPUT_DIR}　に保存されます")

!mkdir -p $OUTPUT_DIR

#@markdown sks is a rare identifier, feel free to replace it.
print(f"[*] 正則化された画像はここに保存されます： {CLASS_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[*]  /content/drive/MyDrive/stable_diffusion_weights/sks　に保存されます
[*] 正則化された画像はここに保存されます： /content/data/yui


In [ ]:
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="princess connect! {CLASS_NAME}" \
  --class_prompt="1girl" \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=0.00000025 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=10000

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--num_cpu_threads_per_process` was set to `1` to improve out-of-box performance
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
The config attributes {'feature_extractor': ['transformers', 'CLIPFeatureExtractor'], 'safety_checker': ['stable_diffusion', 'StableDiffusionSafetyChecker']} were passed to StableDiffusionPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Generating class images: 100% 13/13 [06:43<00:00, 31.02s/it]

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bu

これで学習したdiffusersモデルができました。後は後はckptに変換するだけです。

In [ ]:
#@markdown Download script
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py

In [ ]:
ckpt_path = OUTPUT_DIR + "/model.ckpt"

half_arg = ""
#@markdown  容量がない場合はこちらをクリック。尚多少モデルの質が劣化します
fp16 = False #@param {type: "boolean"}
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $OUTPUT_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

Reshaping encoder.mid.attn_1.q.weight for SD format
Reshaping encoder.mid.attn_1.k.weight for SD format
Reshaping encoder.mid.attn_1.v.weight for SD format
Reshaping encoder.mid.attn_1.proj_out.weight for SD format
Reshaping decoder.mid.attn_1.q.weight for SD format
Reshaping decoder.mid.attn_1.k.weight for SD format
Reshaping decoder.mid.attn_1.v.weight for SD format
Reshaping decoder.mid.attn_1.proj_out.weight for SD format
[*] Converted ckpt saved at /content/drive/MyDrive/stable_diffusion_weights/sks/model.ckpt
